## Soluciones lab06-challenge

Entorno

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, avg, count
from pyspark.sql.types import StringType

# Crear la sesión de Spark
spark = SparkSession.builder.appName("lab06").getOrCreate()

# Cargar datos
trips_df = spark.read.parquet("../../data/yellow_tripdata_2023-01.parquet")
trips_df.show(5)

Apartado 1

In [ ]:
def clasificar_distancia(distance):
    if distance is None:
        return "desconocida"
    elif distance < 2:
        return "corta"
    elif distance < 6:
        return "media"
    else:
        return "larga"

cat_distancia = udf(clasificar_distancia, StringType())

In [ ]:
trips_df.select('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance').withColumn('Cat. distancia', cat_distancia("trip_distance")).show(5)

Apartado 2

In [ ]:
def viaje_dia_noche(date):
    if date is None:
        return "desconocido"
    elif date.hour >= 20 or date.hour < 6:
        return "noche"
    else:
        return "día"

cat_viaje_hora = udf(viaje_dia_noche, StringType())

In [ ]:
trips_df.select('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance').withColumn('Cat. momento día', cat_viaje_hora("tpep_pickup_datetime")).orderBy('tpep_pickup_datetime', ascending=True).show(5)

Apartado 3

In [ ]:
def clasificar_propina(tip_amount):
    if tip_amount is None:
        return 'desconocido'
    elif tip_amount == 0:
        return 'sin propina'
    elif tip_amount < 2:
        return 'baja'
    elif tip_amount > 6:
        return 'alta'
    else:
        return 'media'

cat_propina = udf(clasificar_propina, StringType())

In [ ]:
trips_df.select('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'tip_amount').withColumn('Cat. propina', cat_propina('tip_amount')).show(5)

Apartado 4

In [ ]:
def franja_temp(tpep_pickup_datetime):
    hour = tpep_pickup_datetime.hour
    if hour > 7 and hour < 14:
        return 'mañana'
    elif hour > 14 and hour < 20:
        return 'tarde'
    elif hour > 20:
        return 'noche'
    elif hour < 7:
        return 'madrugada'

cat_franja_temporal = udf(franja_temp, StringType())

def get_profile(distancia_cat, propina_cat, franja_temp):
    if None in (distancia_cat, propina_cat, franja_temp):
        return 'incompleto'
    l = [distancia_cat, propina_cat, franja_temp]
    return '|'.join(l) 

perfil = udf(get_profile, StringType())

In [ ]:
result = trips_df.select('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'tip_amount', 'total_amount').withColumn('Distancia categorizada', cat_distancia('trip_distance')) \
.withColumn('Propina categorizada', cat_propina('tip_amount')) \
.withColumn('Franja temporal', cat_franja_temporal('tpep_pickup_datetime')) \
.withColumn('Perfil', perfil('Distancia categorizada', 'Propina categorizada', 'Franja temporal'))

In [ ]:
result.show(5)

Apartado 5

In [ ]:
print('Perfiles más comunes:')
result.groupBy("Perfil").count().orderBy(col("count").desc()).show()

In [ ]:
print('Promedio de importe total por perfil:')
result.groupBy('Perfil').agg(avg('total_amount').alias('Promedio de importe')).orderBy(col('Promedio de importe').desc()).show()

Apartado 6

In [ ]:
def anomalies(tpep_pickup_datetime, tpep_dropoff_datetime, trip_distance, total_amount, tip_amount, fare_amount):
    try:
        a = 0
        duration = (tpep_dropoff_datetime - tpep_pickup_datetime).total_seconds()/60
        if duration > 120:
            a+=1
        if trip_distance > 30:
            a+=1
        if total_amount > 250:
            a+=1
        if tip_amount > fare_amount:
            a+=1
        return 'sospechoso'if a > 1 else 'normal'
    except:
        return 'desconocido'

viajes_anomalos = udf(anomalies, StringType())

In [ ]:
anomalies = trips_df.withColumn('Tipo de viaje', viajes_anomalos('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'total_amount', 'tip_amount', 'fare_amount'))

In [ ]:
print('Viajes anómalos:')
anomalies.groupBy('Tipo de viaje').count().show()

**Plus**: Realizar dos Joins con el archivo de las zonas de Taxis para mostrar los nombres de las paradas en vez de el ID y mostramos los datos agrupados por recorridos ({PULocation: DOLocation}) para un mejor análisis

In [ ]:
taxi_zones = spark.read.option("header", True).csv("../../data/taxi_zone_lookup.csv")

anomalies_reduced = anomalies.select('tpep_pickup_datetime','tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance', 'fare_amount', 'tip_amount', 'Tipo de viaje')
PU_zones = anomalies_reduced.join(taxi_zones.select('LocationID', 'Zone').withColumnRenamed('LocationID', 'PULocationID').withColumnRenamed('Zone', 'PULocation'), on="PULocationID", how="left")
DO_zones = PU_zones.join(taxi_zones.select('LocationID', 'Zone').withColumnRenamed('LocationID', 'DOLocationID').withColumnRenamed('Zone', 'DOLocation'), on="DOLocationID", how="left")
res = DO_zones.drop('DOLocationID', 'PULocationID')

In [ ]:
print('Análisis de viajes sospechosos por recorrido:')
res.filter(col("Tipo de viaje") == "sospechoso") \
         .groupBy("PULocation", "DOLocation") \
         .count() \
         .orderBy(col("count").desc()) \
         .show()

In [ ]:
spark.stop()